In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob

In [2]:
def preprocZ(ds):
     
    # Not sure how to pass this to a preproc function
    level=850
    lons=[240,360]
    lats=[0,60]
    v='z'
    
    # Extract region and level
    ds=ds.sel(isobaricInhPa=level,latitude=slice(lats[1],lats[0]),longitude=slice(lons[0],lons[1]))

    # Create a new xarray dataset
    ds_tmp=xr.DataArray(ds[v],
                        coords={'lat':ds['latitude'].values,
                                'lon': ds['longitude'].values},
                            dims=['lat','lon'])   
    ds_tmp=ds_tmp.to_dataset(name=v+str(int(level)))
    
    return ds_tmp

In [3]:
def preprocU(ds):
     
    # Not sure how to pass this to a preproc function
    level=850
    lons=[240,360]
    lats=[0,60]
    v='u'
    
    # Extract region and level
    ds=ds.sel(isobaricInhPa=level,latitude=slice(lats[1],lats[0]),longitude=slice(lons[0],lons[1]))

    # Create a new xarray dataset
    ds_tmp=xr.DataArray(ds[v],
                        coords={'lat':ds['latitude'].values,
                                'lon': ds['longitude'].values},
                            dims=['lat','lon'])   
    ds_tmp=ds_tmp.to_dataset(name=v+str(int(level)))
    
    return ds_tmp

In [4]:
# Dates
sdate='19790101'
edate='20181231'

# Years List
yrs_list=np.arange(1979,2019)

In [5]:
# Input
path='/shared/working/rean/era-interim/daily/data/'
fname='ei.oper.an.pl.regn128cm.'

# Output
out_path='/project/predictability/kpegion/wxregimes/era-interim/'
ofname='erai.z850_u850_nash_1979-2019.nc'
outfile=out_path+ofname

In [6]:
# Create empty list to append data for each month
ds_Z_months=[]
ds_U_months=[]

# Loop over months, get list of files and read in data for each month
for m in np.arange(1,13):
        
    # Get all the filenames for this month for all years
    fnamesZ = [f"{path}{year}/{fname}{year}{m:02d}*" for year in yrs_list]       
    print(fnamesZ)
    # Create list of all filenames for this month, and all years
    filesZ=[]
    for files in fnamesZ:
        f2=glob.glob(files)
        for f in f2:
            filesZ.append(f)
    
    filesZ=sorted(filesZ)
    
    # Read Geopotential Height (Z) data           
    ds_Z=xr.open_mfdataset(filesZ,engine='cfgrib',
                           combine='nested',concat_dim=['time'],
                           backend_kwargs={'indexpath':'',
                                           'filter_by_keys':{'name': 'Geopotential'}},
                           preprocess=preprocZ)
 
    # Read zonal wind (U) data
    ds_U=xr.open_mfdataset(filesZ,engine='cfgrib',
                           combine='nested',concat_dim=['time'],
                           backend_kwargs={'indexpath':'',
                                           'filter_by_keys':{'name': 'U component of wind'}},
                           preprocess=preprocU) 

    
    # Create dates for time assign to time dimension
    dates_all=pd.date_range(start=sdate,end=edate,freq='D')
    print(dates_all)
    seas_dates=dates_all[(dates_all.month==m)]
    print(seas_dates)
    ds_Z['time']=seas_dates
    ds_U['time']=seas_dates
        
    # Append the latest month to the list
    ds_Z_months.append(ds_Z)
    ds_U_months.append(ds_U)
        
# Combine the months into the init dimension
ds_Z_months = xr.combine_nested(ds_Z_months, concat_dim='time')
ds_U_months = xr.combine_nested(ds_U_months, concat_dim='time')

['/shared/working/rean/era-interim/daily/data/1979/ei.oper.an.pl.regn128cm.197901*', '/shared/working/rean/era-interim/daily/data/1980/ei.oper.an.pl.regn128cm.198001*', '/shared/working/rean/era-interim/daily/data/1981/ei.oper.an.pl.regn128cm.198101*', '/shared/working/rean/era-interim/daily/data/1982/ei.oper.an.pl.regn128cm.198201*', '/shared/working/rean/era-interim/daily/data/1983/ei.oper.an.pl.regn128cm.198301*', '/shared/working/rean/era-interim/daily/data/1984/ei.oper.an.pl.regn128cm.198401*', '/shared/working/rean/era-interim/daily/data/1985/ei.oper.an.pl.regn128cm.198501*', '/shared/working/rean/era-interim/daily/data/1986/ei.oper.an.pl.regn128cm.198601*', '/shared/working/rean/era-interim/daily/data/1987/ei.oper.an.pl.regn128cm.198701*', '/shared/working/rean/era-interim/daily/data/1988/ei.oper.an.pl.regn128cm.198801*', '/shared/working/rean/era-interim/daily/data/1989/ei.oper.an.pl.regn128cm.198901*', '/shared/working/rean/era-interim/daily/data/1990/ei.oper.an.pl.regn128cm.1

In [7]:
ds_ZU=xr.merge([ds_Z_months,ds_U_months])
ds_ZU

<xarray.Dataset>
Dimensions:  (lat: 86, lon: 170, time: 14610)
Coordinates:
  * lat      (lat) float64 60.0 59.3 58.6 57.89 ... 2.456 1.754 1.053 0.3509
  * lon      (lon) float64 240.5 241.2 241.9 242.6 ... 357.2 357.9 358.6 359.3
  * time     (time) datetime64[ns] 1979-01-01 1979-01-02 ... 2018-12-31
Data variables:
    z850     (time, lat, lon) float32 1.513e+04 1.515e+04 ... 1.48e+04 1.479e+04
    u850     (time, lat, lon) float32 -1.377 -1.861 -2.352 ... -2.985 -3.218

In [8]:
ds_ZU.to_netcdf(outfile)

In [9]:
ds_Z

<xarray.Dataset>
Dimensions:  (lat: 86, lon: 170, time: 1240)
Coordinates:
  * lat      (lat) float64 60.0 59.3 58.6 57.89 ... 2.456 1.754 1.053 0.3509
  * lon      (lon) float64 240.5 241.2 241.9 242.6 ... 357.2 357.9 358.6 359.3
  * time     (time) datetime64[ns] 1979-12-01 1979-12-02 ... 2018-12-31
Data variables:
    z850     (time, lat, lon) float32 1.366e+04 1.366e+04 ... 1.48e+04 1.479e+04

In [10]:
ds_U

<xarray.Dataset>
Dimensions:  (lat: 86, lon: 170, time: 1240)
Coordinates:
  * lat      (lat) float64 60.0 59.3 58.6 57.89 ... 2.456 1.754 1.053 0.3509
  * lon      (lon) float64 240.5 241.2 241.9 242.6 ... 357.2 357.9 358.6 359.3
  * time     (time) datetime64[ns] 1979-12-01 1979-12-02 ... 2018-12-31
Data variables:
    u850     (time, lat, lon) float32 17.52 17.37 16.99 ... -2.779 -2.985 -3.218

In [11]:
seas_dates

DatetimeIndex(['1979-12-01', '1979-12-02', '1979-12-03', '1979-12-04',
               '1979-12-05', '1979-12-06', '1979-12-07', '1979-12-08',
               '1979-12-09', '1979-12-10',
               ...
               '2018-12-22', '2018-12-23', '2018-12-24', '2018-12-25',
               '2018-12-26', '2018-12-27', '2018-12-28', '2018-12-29',
               '2018-12-30', '2018-12-31'],
              dtype='datetime64[ns]', length=1240, freq=None)

In [12]:
print(fnamesZ)

['/shared/working/rean/era-interim/daily/data/1979/ei.oper.an.pl.regn128cm.197912*', '/shared/working/rean/era-interim/daily/data/1980/ei.oper.an.pl.regn128cm.198012*', '/shared/working/rean/era-interim/daily/data/1981/ei.oper.an.pl.regn128cm.198112*', '/shared/working/rean/era-interim/daily/data/1982/ei.oper.an.pl.regn128cm.198212*', '/shared/working/rean/era-interim/daily/data/1983/ei.oper.an.pl.regn128cm.198312*', '/shared/working/rean/era-interim/daily/data/1984/ei.oper.an.pl.regn128cm.198412*', '/shared/working/rean/era-interim/daily/data/1985/ei.oper.an.pl.regn128cm.198512*', '/shared/working/rean/era-interim/daily/data/1986/ei.oper.an.pl.regn128cm.198612*', '/shared/working/rean/era-interim/daily/data/1987/ei.oper.an.pl.regn128cm.198712*', '/shared/working/rean/era-interim/daily/data/1988/ei.oper.an.pl.regn128cm.198812*', '/shared/working/rean/era-interim/daily/data/1989/ei.oper.an.pl.regn128cm.198912*', '/shared/working/rean/era-interim/daily/data/1990/ei.oper.an.pl.regn128cm.1

In [13]:
yrs_list

array([1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989,
       1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000,
       2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011,
       2012, 2013, 2014, 2015, 2016, 2017, 2018])

In [14]:
len(yrs_list)

40

In [15]:
tmp=xr.open_mfdataset(filesZ,engine='cfgrib',
                           combine='nested',concat_dim=['time'],
                           backend_kwargs={'indexpath':'',
                                           'filter_by_keys':{'name': 'Geopotential'}})

In [16]:
tmp

,Array,Chunk
Bytes,22.40 GiB,18.50 MiB
Shape,"(1240, 37, 256, 512)","(1, 37, 256, 512)"
Count,4960 Tasks,1240 Chunks
Type,float32,numpy.ndarray
